# Bayesian amplitude regression

In the lectures we already discussed Bayesian networks.  Here we will learn how to use them in practice.

We will start by constructing a Bayesian layer in pytorch, and then building the Bayesian loss function.  We will then construct a Bayesian network from these layers, and use it to perform the same amplitude regression from the previous tutorial.  We will discuss how to analyse the outputs of the Bayesian network, and how this gives us the ability to estimate the error on our analysis.  This last step is crucial to the application of any numerical technique in physics.

Interesting papers/read:
- Making the most of LHC data - Bayesian neural networks and SMEFT global analysis
    - Michel Luchmann
    - PhD thesis: http://archiv.ub.uni-heidelberg.de/volltextserver/32414/
- Loop Amplitudes from Precision Networks
    - Simon Badger, Anja Butter, Michel Luchmann, Sebastian Pitz, Tilman Plehn
    - https://arxiv.org/abs/2206.14831

#### Outline / tasks:
 - Imports \& plotting set-up
 - Loading the data
 - Preprocessing the data
     - neural networks like $\mathcal{O}(1)$ numbers
     - how should we preprocess the data?
 - Datasets and dataloaders
     - details are in the pytorch docs
 - Building a Bayesian layer
 - Constructing the Bayesian loss function
 - Building the Bayesian neural network
 - Optimizing the neural network
 - Plot the train and validation losses as a function of the epochs
 - Study the results

     
Most practical pytorch skills you need for this work is covered in the basics tutorial at https://pytorch.org/tutorials/beginner/basics/intro.html.

### Imports

In [1]:
import os
import sys
import random
import time
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import Tensor
from torch.nn.parameter import Parameter, UninitializedParameter
from torch.nn import functional as F
from torch.nn import init
from torch.nn import Module

#### Plotting set-up

In [2]:
from matplotlib import pyplot as plt
import matplotlib
import warnings
warnings.filterwarnings("ignore")
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
import matplotlib.colors as mcolors
import colorsys


## Loading the data

In [3]:
trn_dat = np.load("datasets/tutorial-1-data/trn_dat.npy")
trn_amp = np.load("datasets/tutorial-1-data/trn_amp.npy")

val_dat = np.load("datasets/tutorial-1-data/val_dat.npy")
val_amp = np.load("datasets/tutorial-1-data/val_amp.npy")

tst_dat = np.load("datasets/tutorial-1-data/tst_dat.npy")
tst_amp = np.load("datasets/tutorial-1-data/tst_amp.npy")

In [4]:
print(f"train data shape: {trn_dat.shape}")
print(f"train amp  shape: {trn_amp.shape}")
print(f"test  data shape: {tst_dat.shape}")
print(f"test  amp  shape: {tst_amp.shape}")
print(f"val   data shape: {val_dat.shape}")
print(f"val   amp  shape: {val_amp.shape}")

train data shape: (30000, 5, 4)
train amp  shape: (30000,)
test  data shape: (30000, 5, 4)
test  amp  shape: (30000,)
val   data shape: (30000, 5, 4)
val   amp  shape: (30000,)


The data is organised as follows:
 - the shape corresponds to ( number of events, number of particles, 4-momentum )
 - the particles are arranged as follows
     - the first two entries are the two incoming gluons
     - the next two particles are the outgoing photons
     - the last particle is the outgoing gluon
 - the incoming gluons and incoming photons are arranged by transverse memontum $p_T$

## Preprocessing the data

In [ ]:
def get_pt(fv):
    """ returns p_T of given four vector """
    ptsq = np.round(fv[:, 1]**2 + fv[:, 2]**2, 5)
    return np.sqrt(ptsq)

In [ ]:
# index 2 is leading photon, not gluon (which is 4)
trn_dat_gluon_pt = get_pt(trn_dat[:, 4])

In [7]:
nev = trn_dat.shape[0]
trn_datf = np.reshape(trn_dat, (nev,-1))
val_datf = np.reshape(val_dat, (nev,-1))
tst_datf = np.reshape(tst_dat, (nev,-1))

In [8]:
trn_datf.shape

(30000, 20)

In [9]:
gpt = np.mean(trn_dat_gluon_pt)
trn_datf = trn_datf / gpt
val_datf = val_datf / gpt
tst_datf = tst_datf / gpt

In [10]:
trn_datfp = torch.Tensor(trn_datf)
val_datfp = torch.Tensor(val_datf)
tst_datfp = torch.Tensor(tst_datf)

In [11]:
trn_ampl = np.log(trn_amp)
val_ampl = np.log(val_amp)
tst_ampl = np.log(tst_amp)

In [12]:
trn_amplp = torch.Tensor(trn_ampl)
val_amplp = torch.Tensor(val_ampl)
tst_amplp = torch.Tensor(tst_ampl)

## Datasets and dataloaders

In [13]:
class amp_dataset(Dataset):
    
    def __init__(self, data, amp):
        self.data = data
        self.amp = amp

    def __len__(self):
        return len(self.amp)

    def __getitem__(self, idx):
        return self.data[idx], self.amp[idx]

In [14]:
trn_dataset = amp_dataset(trn_datfp, trn_amplp.unsqueeze(-1))
val_dataset = amp_dataset(val_datfp, val_amplp.unsqueeze(-1))
tst_dataset = amp_dataset(tst_datfp, tst_amplp.unsqueeze(-1))

In [15]:
trn_dataloader = DataLoader(trn_dataset, batch_size=64, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)
tst_dataloader = DataLoader(tst_dataset, batch_size=64, shuffle=True)

## Building a Bayesian layer

First let's look at the source code for a **basic linear layer** in pytorch:

(https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear)

```
class Linear(Module):
    
    __constants__ = ['in_features', 'out_features']
    in_features: int
    out_features: int
    weight: Tensor

    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(Linear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        if bias:
            self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input: Tensor) -> Tensor:
        return F.linear(input, self.weight, self.bias)

    def extra_repr(self) -> str:
        return 'in_features={}, out_features={}, bias={}'.format(
            self.in_features, self.out_features, self.bias is not None
        )
```

Objects of this class apply a linear transformation to the incoming data: $y = xA^T + b$.

The input arguments are required to initialise the layer, so, in the \_\_init()\_\_ function.  We have:
- in_features: size of each input sample
- out_features: size of each output sample
- bias: If set to ``False``, the layer will not learn an additive bias.  Default: ``True``

The shapes are:
- Input: $(*, H_{in})$ where $*$ means any number of dimensions including none and $H_{in} = \text{in-features}$.
- Output: $(*, H_{out})$ where all but the last dimension are the same shape as the input and $H_{out} = \text{out-features}$.

The layer has attributes:
- weight: the learnable weights of the module of shape $(\text{out-features}, \text{in-features})$. The values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$, where $k = \frac{1}{\text{in-features}}$
- bias:   the learnable bias of the module of shape $(\text{out-features})$.  If `bias` is ``True``, the values are initialized from $\mathcal{U}(-\sqrt{k}, \sqrt{k})$ where $k = \frac{1}{\text{in-features}}$.

Examples::

    >>> m = nn.Linear(20, 30)
    
    >>> input = torch.randn(128, 20)
    
    >>> output = m(input)
    
    >>> print(output.size())
    
    torch.Size([128, 30])

From the lecture, we know that in a Bayesian network the weights are replaced by Gaussian distributions, and on a forward pass we get the output by sampling from that distribution.

So the biases are the same as in the linear layer.  But not each weight is a Gaussian distibution and so needs a mean and a variance.  The bias and the mean and variance of the weight distribution will be learnable.  In practice it's easier to work with the log of the variance as it is more stable when optimising the network.

We need to be able to sample from the Gaussian weight distributions, and compute derivatives of the output in order to update the network parameters.  To do this we use something called the 're-parameterisation trick'.  It involves sampling random noise from a unit normal distribution, and then transforming that number using the mean and variance of the weight distribution in this way:
\begin{equation}
w = \mu + \sigma\times r
\end{equation}
where $r$ is a random number sampled from a unit normal distribution (Gaussian with mean and variance equal to one), $\mu$ is the mean of the weight distribution, and $\sigma$ is the standard deviation.  In this way we separate the stochastic part of the function from the parameters defining the distribution.  And so if we take any differentiable function of $x$ (e.g. an activation function), we can compute derivatives of that function with respect to the mean and variance of the weight distribution.

In the `forward` method we then need to implement this reparameterisation trick for the weights, and then apply the same linear transformation as in the standard linear layer.

On each forward pass we need to generate a set of random numbers with the same shape as our means and variances.  Choosing a set of random numbers for the sampling is equivalent to 'sampling' a new neural network from the Bayesian neural network.  And sometimes at the end of the analysis, we will want to keep the same network for testing.  So we don't always want to re-sample the random numbers on each forward pass.  To control this we define a flag called `self.resample`, with default set to ``True``.

We also need a `reset_parameters` function to reset the parameters in the network.  This is standard for layers in pytorch.  We use a slightly different function to do this than is used in the pytorch linear layer, as can be seen below.

From the lecture, you know that the weight distributions require a prior.  The simplest choice for this prior is just a Gaussian distribution with a mean and variance of one.  Results are typically not too sensitive to this prior, as long as the values are within reasonable limits.  For example, $\mathcal{O}(1)$ means and standard deviations.  Going beyond $\mathcal{O}(1)$ numbers just leads to numerical instabilities in the training.  The Bayesian loss function contains a term coming from the KL divergence between the weight distributions in the network and their priors.  So the layers should have some funcitonality to return these values.  The KL divergence for the layer is:
\begin{equation}
\text{KL} = \sum_{\text{weights}} 0.5 \times (  \mu^2 + \sigma^2 - \log\sigma^2 - 1 )
\end{equation}

So let's build the **simplest Bayesian layer** we can.

Make sure you include a KL divergence for the layer

In [ ]:
#TODO: Build the layer

Test the layer and verify that running the same input multiple times yields different outputs

In [ ]:
#TODO

## The Bayesian loss function

From the lectures we know that there are two parts to the Bayesian loss function:
- The negative log Gaussian
- the KL from the network weights

The second comes from the layers, and the first is defined below.  This negative log Gaussian term acts on the two outputs from the Bayesian neural network:
- the mean
- the variance (which we parameterise as the log variance)

In [ ]:
# TODO: Build the loss function (neg_log_gauss)

## Building the Bayesian neural network

We'll build a simple network with one input and one output layer, and two hidden layers.  We define the dimensions of these layers below.

In [23]:
ipt_dim = #input_dim
opt_dim = #output_dim
hdn_dim = #input_dim

Now we build a very simple class for our neural network, which we call amp_net.

In [ ]:
#TODO: define the bayesian regression net

Initialise the neural network and send to the GPU if we have one.  We can also print the model to see what layers it has.

In [ ]:
#TODO: actually build the network

The predicitons of the full Bayesian network should vary with each forward pass because of the weight sampling

In [ ]:
#TODO: check this is true

## Optimizing (training) the neural network

The Bayesian loss function has two terms which we have already definedl; the negative los Gaussian, and the KL divergence between the network and the network prior.  The latter comes from the KL divergence over the weights in the network.

Now we can write a training loop for a single epoch.

In [31]:
def train_epoch(dataloader, model, optimizer):

    size = len(dataloader.dataset)
    model.train()
    loss_tot, kl_tot, neg_log_tot = 0.0, 0.0, 0.0
    loss_during_opt, kl_during_opt, neg_log_during_opt = 0., 0., 0.
    
    for batch, (X, y) in enumerate(dataloader):
        
        # pass data through network
        pred = model(X)
        
        # compute loss
        nl = neg_log_gauss(pred, y.reshape(-1))
        kl = model.KL()
        loss = nl + kl

        loss_during_opt += loss.item()
        kl_during_opt += kl.item()
        neg_log_during_opt += nl.item()

        # reset gradients in optimizer
        optimizer.zero_grad()
        
        # compute gradients
        loss.backward()
        
        # update weights with optimizer
        optimizer.step()
        
        # print the training loss every 100 updates
        if batch % 100 == 0:
            current = batch * len( X )
            print(f"current batch loss: {loss:>8f} KL: {kl:>8f} Neg-log {nl:>8f}  [{current:>5d}/{size:>5d}]")
    loss_live = loss_during_opt/len(dataloader)
    kl_live = kl_during_opt / len(dataloader)
    nl_live = neg_log_during_opt / len(dataloader)
    
    print(f"avg train loss per batch in training: {loss_live:>8f}")        
    return loss_live, kl_live, nl_live

To monitor the performance of the network on the regression task we want to calculate the loss of both the training data and the validation data on the same network, so we have the following functions:

In [32]:
def val_pass(dataloader, model):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    nls = 0.0
    kls = 0.0
    vls = 0.0
    mse = 0.0

    # we don't need gradients here since we only use the forward pass
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            nl = neg_log_gauss(pred, y.reshape(-1))
            kl = model.KL()
            vl = nl.item() + kl.item()
            mse += torch.mean((pred[:, 0] - y.reshape(-1))**2)
            nls += nl
            kls += kl
            vls += vl

    nls /= num_batches
    kls /= num_batches
    vls /= num_batches
    mse /= num_batches
    print( f"avg val loss per batch: {vls:>8f} KL: {kls:>8f} Neg-log {nls:>8f} MSE {mse:>8}" )
    
    return nls.cpu().numpy(), kls.cpu().numpy(), vls, mse.cpu().numpy()

def trn_pass(dataloader, model):
    
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    nls = 0.0
    kls = 0.0
    tls = 0.0
    mse = 0.0

    # we don't need gradients here since we only use the forward pass
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            nl = neg_log_gauss(pred, y.reshape(-1))
            kl = model.KL()
            tl = nl.item() + kl.item()
            mse += torch.mean((pred[:, 0] - y.reshape(-1))**2)
            nls += nl
            kls += kl
            tls += tl

    nls /= num_batches
    kls /= num_batches
    tls /= num_batches
    mse /= num_batches
    print( f"avg trn loss per batch: {tls:>8f} KL: {kls:>8f} Neg-log {nls:>8f} MSE {mse:>8}" )
    
    return nls.cpu().numpy(), kls.cpu().numpy(), tls, mse.cpu().numpy()

Now we can train the model!

In [ ]:
# TODO: Write the training loop

## Plot the train and validation losses as a function of the epochs

In [ ]:
# TODO: plot the losses

## Study the results

Now we want to get some visualisation of how well our amplitude regression has worked.

The simplest thing we can do is to pass our data through the neural network to get a predicted amplitude for each event, then histogram this and compare it to the histogram of the true amplitudes.

In [ ]:
# TODO